# Finding Repeats
## Nov 2020

DNA often has repeats, due to the activity of transposons.

Repeats complicate the task of sequencing the genome, which is done by piecing together short subsequences. 
If the repeat is longer than the typical subsequence, it is difficult to know which copy of a repeat a sequence preceeds or follows.  

### The problem

Given a string 'text', we are looking for a tripple t = (length, pos1, pos2) such that pos1 != pos2 and

```python
    text[pos1:pos1 + length] == text[pos2:pos2 + length] 
```

### Special Cases

There are Tandem Repeats, where pos2 ~ pos1 + length, such as 'Duran Duran'.   

```python
    Duran Duran
          Duran Duran
```

(See Murmur at the poetry slam: http://www.ironicsans.com/2007/05/murmur_at_the_poetry_slam.html)

There are overlap repeats such as AGAGAGAGAG inside AGAGAGAGAGAG, where pos1 = 0, pos2 = 2 and length = 10

```python
    AGAGAGAGAGAG
      AGAGAGAGAG AG
```

## Algorithm 1

Iterate over all pairs pos1, pos2, with pos1 < pos2, and compare the strings

*We can move the starting point for pos2 to avoid overlap*

Remember the largest match seen

```python
    for pos1 in range(len(text)):
        for pos2 in range(pos1 + 1, len(text)):
            find largest length such that text[pos1:pos1 + length] == text[pos2:pos2 + length]
            If this is greater than current max, remember it
```
We are looking at N^2 spots, and testing in each spot

## Algorithm 2

For each position, pos1, see how long a mach we can find

```python
    for pos1 in range(len(text)):
        increase ln until we cannot find a match in text[pos1+1:] 
        If this is greater than current max, remember it
```

We can use Python's 'in' and 'find()' to speed this up.

We are looking at N starting positions, and searching for a matchin sequence.

No need to search for strings shorter than what we've already found.

## Algorithm 3

This is a special case of the Smith-Waterman ocal alignment problem. 

See the figure here: https://en.wikipedia.org/wiki/Smith–Waterman_algorithm

We build an array holding the length of a match.  For example, look for repeats in the string "AGGCTAGCT"
    
We are looking for large values off the diagonal - I've bumped the longest match (3) so it will pop out.  

```python
          A  G  G  C  T  A  G  C  T  
       0  0  0  0  0  0  0  0  0  0
    A  0  1  0  0  0  0  1  0  0  0
    G  0  0  2  1  0  0  0  2  0  0
    G  0  0  1  3  0  0  0  1  0  0
    C  0  0  0  0  4  0  0  0  2  0
    T  0  0  0  0  0  5  0  0  0   3
    A  0  1  0  0  0  0  6  0  0  0
    G  0  0  2  1  0  0  0  7  0  0
    C  0  0  0  0  2  0  0  0  8  0
    T  0  0  0  0  0  3  0  0  0  9
```

## The code fragment below shows how to deal with a 2-D array, stored as a list of lists

In [ ]:
# Pretty Print the array
#     table      - array holding tableau to print
#     str1, str2 - strings being aligned
def printArray(table, str1, str2):

    # Print the column headings
    print("   ", end = '')         # Add spacing
    for c in range(len(str2)):
        print(str2[c], " ", end = '')    # Try to match field below
    print()

    # Print the contents
    for r in range(len(str1)):

        # Print the row headings
        print(str1[r], end= '')

        # Print the row
        for c in range(len(str2)):
            print(repr(table[r][c]).rjust(3), end= '')   # Field width of 3
        print()                # Newline
    print()                    # Blank line at end

### Just look at top right

Focus on the array: align "AGGCTAGCT"
    
We are looking for large values off the diagonal

```python
      A  G  G  C  T  A  G  C  T  
      0  0  0  0  0  0  0  0  0
A        0  0  0  0  1  0  0  0
G           1  0  0  0  2  0  0
G              0  0  0  1  0  0
C                 0  0  0  2  0
T                    0  0  0  3
A                       0  0  0
G                          0  0
C                             0
T                            
```

### Prune further

Focus on the array: align "AGGCTAGCT"
    
We are looking for large values off the diagonal

Focus on the key parts

```python
                     A  G  C  T  
      
A                    1 
G                    0  2  
G                       1 
C                          2  
T                             3                         
```

## Algorithm 4

Algorithm 3 creates a large matrix

We don't need the full matrix: can work with current row and previous row.

## Algorithm 5 - The Suffix Tree 

Excellent algorithm, but the Data structure is too complicated to discuss in this course.  
    
However, we can use the ideas in the next algorithm.  

## Algorithm 6: Sort and Search

Take our string

```python
    text = "AGGCTAGCT"
```

Build a list of suffixes of the string

```python
    lst = [ text[i:] for i in range(len(text))]
```

In [ ]:
text = "AGGCTAGCT"
lst = [ text[i:] for i in range(len(text))]
for item in lst:
    print(item)

## Sort the list

In [ ]:
text = "AGGCTAGCT"
lst = [ text[i:] for i in range(len(text))]
for item in sorted(lst):
    print(item)

## Look at adjacent strings, and find best initial match
```python
    AG CT
    AG GCTAGCT
    CT
    CT AGCT
    GCT
    GCT AGCT
    GGCTAGCT
    T
    T AGCT
```

## Drawbacks

The list of suffixes has Quadratic size

*Don't really need the strings: all we need is the index to the first item*

Hard to sort based on that representation.  Python 2 let you define a cmp() function - there are workarounds in Python 3

In [ ]:
text = 'AACCAACCCTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG'

In [ ]:
lst = [ text[i:] for i in range(len(text))]
for item in sorted(lst):
    print(item)

## Scroll down for a match of length 6

```python
    AAC ATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    AACC AACCCTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    AACC CTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ACC AACCCTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ACC CTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ACGGGTG
    AGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    AGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    AGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    AGCGTTGGGTCCTGGCCACGGGTG
    ATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ATC CATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ATC GCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ATCTC CAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    ATCTC GGGCAGCGTTGGGTCCTGGCCACGGGTG
    CAACCCTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CACGGGTG
    CAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CAGCGTTGGGTCCTGGCCACGGGTG
    CATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
    CATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG
 
```

## Algorithm 7: Doubling Table

Build a Dictionary mapping sequences of a fixed length to the positions seen in the string.

In [2]:
text = 'AACCAACCCTTGGCAGAACATATCCATCGCGTCCGCCATCTCCAGCAGCCGCACGCGGCGCATCTCGGGCAGCGTTGGGTCCTGGCCACGGGTG'

In [5]:
from collections import defaultdict

d = defaultdict(list)

for i in range(len(text)):
    d[text[i:i+5]].append(i)
    
for key in d:
    print(key, d[key])

AACCA [0]
ACCAA [1]
CCAAC [2]
CAACC [3]
AACCC [4]
ACCCT [5]
CCCTT [6]
CCTTG [7]
CTTGG [8]
TTGGC [9]
TGGCA [10]
GGCAG [11, 67]
GCAGA [12]
CAGAA [13]
AGAAC [14]
GAACA [15]
AACAT [16]
ACATA [17]
CATAT [18]
ATATC [19]
TATCC [20]
ATCCA [21]
TCCAT [22]
CCATC [23, 35]
CATCG [24]
ATCGC [25]
TCGCG [26]
CGCGT [27]
GCGTC [28]
CGTCC [29]
GTCCG [30]
TCCGC [31]
CCGCC [32]
CGCCA [33]
GCCAT [34]
CATCT [36, 60]
ATCTC [37, 61]
TCTCC [38]
CTCCA [39]
TCCAG [40]
CCAGC [41]
CAGCA [42]
AGCAG [43]
GCAGC [44, 68]
CAGCC [45]
AGCCG [46]
GCCGC [47]
CCGCA [48]
CGCAC [49]
GCACG [50]
CACGC [51]
ACGCG [52]
CGCGG [53]
GCGGC [54]
CGGCG [55]
GGCGC [56]
GCGCA [57]
CGCAT [58]
GCATC [59]
TCTCG [62]
CTCGG [63]
TCGGG [64]
CGGGC [65]
GGGCA [66]
CAGCG [69]
AGCGT [70]
GCGTT [71]
CGTTG [72]
GTTGG [73]
TTGGG [74]
TGGGT [75]
GGGTC [76]
GGTCC [77]
GTCCT [78]
TCCTG [79]
CCTGG [80]
CTGGC [81]
TGGCC [82]
GGCCA [83]
GCCAC [84]
CCACG [85]
CACGG [86]
ACGGG [87]
CGGGT [88]
GGGTG [89]
GGTG [90]
GTG [91]
TG [92]
G [93]


## What does this mean?

```python
    TGGCA [10]
    GGCAG [11, 67]
```

This says that text[10:15] is TGGCA, and GGCAG appears twice: once as text[11:16] and once as text[67:72]

## Look at the repeats

In [6]:
for key in d:
    if len(d[key]) > 1:
        print(key, d[key])

GGCAG [11, 67]
CCATC [23, 35]
CATCT [36, 60]
ATCTC [37, 61]
GCAGC [44, 68]


## There are 5 repeats of length 5

```python
    GGCAG [11, 67]
    CCATC [23, 35]
    CATCT [36, 60]
    ATCTC [37, 61]
    GCAGC [44, 68]
```

One of them is two views of a repeat of length 6

```python
    CATCT [36, 60]
    ATCTC [37, 61]
    
    CATCTC
```

## Idea

We create a sequence of dictionaries with keys of increasing length

At each stage we can elimate keys that only appear once, such as 'GCCAT' above.

This gives us a smaller set of candidates to investigate

There is a tradeoff here: the longer the key, the faster we can winnow out the set.
However, longer keys require more storage.  

## Note: this was by far the fastest approach of the ones I tried.  